# Analysis 

## Libraries 

In [7]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Data set & Preprocessing

**Import Data set**

In [11]:
#file = 'Data/data.csv'
file = 'Data/discrimant_tweet_47368_dataset.csv'
dataframe = pd.read_csv(file, error_bad_lines=False, sep=";")
dataframe = dataframe[['Tweets','Label']]

# Data set for a supervised learning 
df = dataframe[dataframe['Label'].isnull()==False]

print("Data set's shape = ", df.shape) 
print("There is {} missing values in the target".format(df['Label'].isnull().sum()))

Data set's shape =  (19340, 2)
There is 0 missing values in the target


**Data Cleaning**

In [15]:
from Vectorizer import preprocessing 
#X_train, X_test, y_train, y_test = preprocessing(df[:1000], 20, "bow")

          Tokens  Frequencies
0          video            7
1      peshmerga            6
2     decimating            1
3           isis           26
4            far           10
...          ...          ...
3353      feicin            1
3354  hillarious            1
3355         leo            1
3356        dies            1
3357       grabs            1

[3358 rows x 2 columns]
0.0    679
1.0    321
Name: Label, dtype: int64


In [16]:
from packages.Models import test
test(2)

3